In [11]:
%gui qt
from MUA import *
from xview import ObjectWidget, view_multiwave, view_scatter_3d
from phy.plot import View
from phy.plot.interact import Grid
from hdbscan import HDBSCAN
import phy
from phy.gui import GUI
import seaborn as sns

In [2]:
filename  = 'S:/pcie.bin'
nCh       = 32
fs        = 25000
numbyte   = 4
time_span = 1 # 1 seconds
global time_slice
time_slice = 0
span = time_span * fs
highlight = True

In [3]:
mua = MUA(filename=filename, nCh=nCh, fs=fs, numbytes=numbyte)
spk = mua.tospk()

In [4]:
fet = spk.tofet('pca')

In [5]:
gui = GUI(position=(0, 0), size=(600, 400), name='GUI')

In [6]:
props = ObjectWidget()
gui.add_view(props,position='left', name='params')

In [7]:
scatter_view = view_scatter_3d()
scatter_view.unfreeze()
gui.add_view(scatter_view)
spk_view = View('grid')
gui.add_view(spk_view)

In [8]:
wave_view = view_multiwave(mua.data[time_slice*span:(time_slice+1)*span,:], fs=25000)
wave_view.gap_value = 0.9
wave_view.attach(gui)

In [9]:
from phy.gui import Actions
actions = Actions(gui)

@actions.add(shortcut=',')
def page_up():
#     print 'page_up'
    global time_slice
    time_slice -= 1
    if time_slice >= 0:
        wave_view.set_data(mua.data[time_slice*span:(time_slice+1)*span,:], time_slice)
    else:
        time_slice = 0

@actions.add(shortcut='.')
def page_down():
#     print 'page_down'
    global time_slice
    time_slice += 1
    wave_view.set_data(mua.data[time_slice*span:(time_slice+1)*span,:], time_slice)

In [10]:
gui.show()

In [11]:
gui.list_views()

 <view_scatter_3d (PyQt4) at 0xcb2e208L>,
 <View (PyQt4) at 0xd797cf8L>,
 <view_multiwave (PyQt4) at 0xd6343c8L>]

In [12]:
def update_params():
    ch = props.ch.value()
    fet_method = props.fet_combo.currentText()
    if fet_method == 'peak':
        fet = spk.tofet('peak')
    elif fet_method == 'pca':
        fet = spk.tofet('pca')
        
    min_cluster_size = 20
    #     min_samples = 3
    leaf_size = 30
    hdbcluster = HDBSCAN(min_cluster_size=min_cluster_size, 
                     leaf_size=leaf_size,
                     gen_min_span_tree=True, 
    #                      min_samples = min_samples,
                     algorithm='best')
    global clu
    clu = hdbcluster.fit_predict(fet[ch])    
    nclu = len(np.unique(clu))
    palette = sns.color_palette()    
    
    # scatter_view update
    scatter_view.set_data(fet=fet[ch], clu=clu)
    
    # spk_view update
    spk_view.clear()
    spk_view.grid.shape = (3, nclu)
    for chNo in range(3):
        for clu_id in np.unique(clu):
            color = palette[clu_id] if clu_id>=0 else np.array([1,1,1])
            spk_view[chNo,clu_id+1].plot(y = spk[ch][clu==clu_id,:,chNo].squeeze(),
                                                  color = np.hstack((color, 0.2)),
                                                  data_bounds=(-2, -1800, 2, 800))
    spk_view.build()

    # wave_view update
    highlight_list = mua.spk_peak_pos
    global time_slice
    sublist = np.logical_and(highlight_list[:,0]<(time_slice+1)*span, 
                             highlight_list[:,0]>(time_slice)*span)
    mask_n = highlight_list[sublist][:,0] - time_slice*span
    mask_ch = highlight_list[sublist][:,1]
    mask = np.hstack((mask_n.reshape(-1,1), mask_ch.reshape(-1,1)))

    wave_view.waves1.highlight_cancel()
    for clu_id in np.unique(clu):
        color = palette[clu_id] if clu_id>=0 else np.array([1,1,1])
        wave_view.waves1.highlight_spikes(mask[np.logical_and(clu==clu_id, mask_ch=ch)], color = np.hstack((color, 1)))
    

props.signal_objet_changed.connect(update_params)

In [12]:
def get_current_spikes(ch=None):
    highlight_list = mua.spk_peak_pos
    global time_slice
    sublist = np.logical_and(highlight_list[:,0]<(time_slice+1)*span, 
                             highlight_list[:,0]>(time_slice)*span)
    mask_n = highlight_list[sublist][:,0] - time_slice*span
    mask_ch = highlight_list[sublist][:,1]
    mask = np.hstack((mask_n.reshape(-1,1), mask_ch.reshape(-1,1)))
    
    if ch is None:
        return mask
    else:
        mask = mask[mask[:,1]==ch]
        return mask

In [33]:
ch = props.ch.value()
mask = get_current_spikes(ch)

In [34]:
wave_view.waves1.highlight_cancel()
wave_view.waves1.highlight_spikes(mask, color=(0,1,1,0.5))

In [39]:
wave_view.waves1.highlight(spacial_code=(31,), 
                           temporal_code=((0,span),), 
                           highlight_color=(1,1,1,.2))

In [41]:
gui.show()

# Manual

In [11]:
from phy.cluster.manual import Clustering
from phy.cluster.manual.gui_component import ManualClustering

In [12]:
clu += 1
clustering = Clustering(clu)

In [13]:
clustering.cluster_ids

array([0, 1, 2], dtype=int64)

In [23]:
spike_clusters = clu
spikes_per_cluster = lambda c: np.where(clu==c)[0]

In [24]:
spikes_per_cluster(0)

array([ 338,  373,  374,  384,  420,  453,  496,  560,  578,  618,  634,
        663,  764,  768,  795, 1118, 1278, 1288, 1327, 1328, 1444, 1528,
       1672, 1821, 1922, 1998, 1999, 2119, 2145, 2180, 2268, 2432, 2445,
       2644, 2895, 2940, 2963, 3061, 3099, 3127, 3223, 3253, 3392, 3427,
       3440, 3475, 3799, 3846, 3858, 3873, 3913, 3930, 3973, 3992, 4223,
       4352, 4387, 4408, 4544, 4599, 4774, 4789, 4812, 4912, 5008, 5009,
       5026, 5077, 5207, 5268, 5286, 5360, 5460, 5498, 5538, 5572, 5592,
       5684, 5692, 5700, 5720, 5729, 5739, 5761, 5781, 5818, 5869, 5888,
       5889, 5896, 5898, 5902, 5961, 5962, 5987, 6052, 6084, 6122, 6129,
       6142, 6188, 6200, 6211, 6235, 6312, 6315, 6355, 6374, 6444, 6450,
       6459], dtype=int64)

In [25]:
mc = ManualClustering(spike_clusters, spikes_per_cluster)

In [26]:
mc.attach(gui)

# Table

In [11]:
from phy.gui import Table

In [12]:
table = Table()

In [13]:
@table.add_column
def name(id):
    # This function takes an id as input and returns a string.
    return "My id is %d" % id

In [14]:
table.set_rows([2, 3, 5, 7])

In [15]:
@table.connect_
def on_select(ids):
    print("The items %s have been selected." % ids)

In [17]:
gui.add_view(table)

In [18]:
table.show()

The items [2] have been selected.
The items [3] have been selected.
The items [5] have been selected.
The items [5, 7] have been selected.


In [19]:
from phy.gui import Actions

In [20]:
actions = Actions(gui)

In [21]:
@actions.add(shortcut='c')
def respond():
    print 'c'

The items [5] have been selected.
c
c
The items [3] have been selected.
The items [3, 7] have been selected.
c
c


In [49]:
gui.show()